In [1]:
import os
import glob
import json
import pandas
from tqdm import tqdm 
from collections import defaultdict
from datetime import datetime, timedelta
from src.models.video.labels import EVENT_DICTIONARY_V2
from src.preprocessing.video import(
    json_reader,
    round_down_to_minute,
    round_down_to_minute_v2,
    normalize_path,
    round_down_to_minute_half
)

In [3]:
EVENT_DICT = {key: index for index, key in enumerate(EVENT_DICTIONARY_V2.keys())}
EVENT_DICT
video_path = os.listdir("F:/video_classification")
video_paths = [f"F:/video_classification/{i}" for i in video_path]

labels = [normalize_path(glob.glob(f"F:/video_classification/{i}/*.json")[0]) for i in video_path]
labels

['F:/video_classification/2014-11-04 - 22-45 Arsenal 3 - 3 Anderlecht/Labels-v2.json',
 'F:/video_classification/2014-11-04 - 22-45 Dortmund 4 - 1 Galatasaray/Labels-v2.json',
 'F:/video_classification/2014-11-05 - 22-45 Bayern Munich 2 - 0 AS Roma/Labels-v2.json',
 'F:/video_classification/2014-12-09 - 22-45 Benfica 0 - 0 Bayer Leverkusen/Labels-v2.json',
 'F:/video_classification/2014-12-09 - 22-45 Dortmund 1 - 1 Anderlecht/Labels-v2.json',
 'F:/video_classification/2014-12-09 - 22-45 Liverpool 1 - 1 Basel/Labels-v2.json',
 'F:/video_classification/2014-12-10 - 22-45 Barcelona 3 - 1 Paris SG/Labels-v2.json',
 'F:/video_classification/2014-12-10 - 22-45 Bayern Munich 3 - 0 CSKA Moscow/Labels-v2.json',
 'F:/video_classification/2015-02-15 - 14-30 AC Milan 1 - 1 Empoli/Labels-v2.json',
 'F:/video_classification/2015-02-17 - 22-45 Paris SG 1 - 1 Chelsea/Labels-v2.json',
 'F:/video_classification/2015-02-17 - 22-45 Shakhtar Donetsk 0 - 0 Bayern Munich/Labels-v2.json',
 'F:/video_classific

In [6]:
import json

with open(labels[0], 'r') as f:
    data = json.load(f)

for anno in data['annotations']:
    if anno['label'] == "Kick-off":
        print(anno)

{'gameTime': '1 - 00:00', 'label': 'Kick-off', 'position': '23', 'team': 'home', 'visibility': 'not shown'}
{'gameTime': '1 - 25:08', 'label': 'Kick-off', 'position': '1508964', 'team': 'away', 'visibility': 'visible'}
{'gameTime': '1 - 28:49', 'label': 'Kick-off', 'position': '1729221', 'team': 'away', 'visibility': 'not shown'}
{'gameTime': '2 - 00:00', 'label': 'Kick-off', 'position': '3', 'team': 'away', 'visibility': 'not shown'}
{'gameTime': '2 - 13:39', 'label': 'Kick-off', 'position': '819068', 'team': 'away', 'visibility': 'not shown'}
{'gameTime': '2 - 16:26', 'label': 'Kick-off', 'position': '986030', 'team': 'home', 'visibility': 'not shown'}
{'gameTime': '2 - 27:41', 'label': 'Kick-off', 'position': '1661488', 'team': 'home', 'visibility': 'not shown'}
{'gameTime': '2 - 45:39', 'label': 'Kick-off', 'position': '2739708', 'team': 'home', 'visibility': 'not shown'}


In [5]:
NO_NEEDED_CLASSES = []
for games in tqdm(video_paths):
    # print(games)
    anno_path = normalize_path(os.path.join(games, 'Labels-v2.json'))
    annos = json_reader(anno_path)['annotations']
    # Bỏ qua các frames mà không được show
    # tức là các frame này khả năng cao không có ý nghĩa
    
    # annos_v2 = [
    #     item for item in annos 
    #     if item['visibility'] != 'not shown' 
            # and item['label'] not in NO_NEEDED_CLASSES
        # ]
    
    # annos_v3 = [item for item in annos_v2
    #     if item['label'] == 'Kick-off' and (item['gameTime'].startswith('1 - 00:') 
    #                                     or item['gameTime'].startswith('2 - 00:'))      
        # ]
    # print(annos_v3)
    
    grouped_items = defaultdict(list)
    annos_v2 = sorted([
        dict(t) for t in {tuple(d.items()) for d in annos}], 
        key = lambda x: x['gameTime']
        )
    
    for item in annos_v2:
        if item['gameTime'].startswith("1"):
            group = round_down_to_minute(item['gameTime'].split(' - ')[-1])
            # group2 = round_down_to_minute_half(item['gameTime'].split(' - ')[-1], delta = 0)
        else:
            group = round_down_to_minute_v2(item['gameTime'].split(' - ')[-1])
            # group2 = round_down_to_minute_half(item['gameTime'].split(' - ')[-1], delta = 45)
        grouped_items[group].append(item['label'])
        # grouped_items[group2].append(item['label'])
    grouped_items = defaultdict(list, {k: v for k, v in grouped_items.items() if not k.startswith("-")})
    # sorted keys chỉ chứa các thông tin về các phút chứa sự kiện
    # tức là có thể có CÁC PHÚT không có sự kiện nào xảy ra
    # sort_keys là các time-minute 
    sorted_keys = sorted(
            grouped_items.keys(), 
            key = lambda x: int(x.split(":")[0])
        )
    result = {}
    previous_minute = None
    
    for key in sorted_keys:
        current_minute = int(key.split(":")[0])
        if previous_minute is not None and current_minute > previous_minute +1:
            for missing_minute in range(previous_minute + 1, current_minute):
                # Xử lý trường hợp có phút bị khuyết thì bổ sung thêm
                # Và đặt nó thành list rỗng
                result[f"{missing_minute:02}:00"] = ["Event not recognized"]      
        result[key] = grouped_items[key]
        previous_minute = current_minute
    # # multilabel
    result = {k: list(set(v)) for k, v in result.items()}
    # print(result)
    # print("----")
    # create initial dataset
    df = pandas.DataFrame({
        'chunk': [int(i.split(":")[0]) for i in result.keys()],
        'video_base_dir': games,
        "timestamp": result.keys(),
        "labels": [values for values in result.values()],  # Join list items into a single string
    })
    
    df['labels_encoder'] = df['labels'].apply(
        lambda label_list: ", ".join(
            str(v) for k, v in EVENT_DICT.items() if k in label_list
        ) if label_list else str(list(EVENT_DICT.values())[-1])
    )

    df['video_path']= df.apply(
        lambda row: glob.glob(f"{row['video_base_dir']}/chunk_{int(row['timestamp'].split(":")[0])}/*.mp4"), 
        axis=1
    )
    
    # df['video_path'] = [normalize_path(i[0]) for i in df['video_path']]
    
    df = df[['chunk','video_base_dir', 'video_path', 'timestamp', 'labels', 'labels_encoder']]
    
    # vdo_paths = []
    # ado_paths = []
    # for video in df['video_path']:
    #     vdo_path = f"{video}/visual"
    #     ado_path = f"{video}/audio"
    #     vdo_paths.append(vdo_path)
    #     ado_paths.append(ado_path)

    # df['new_video_path'] = vdo_paths
    # df['new_audio_path'] = ado_paths
    df.to_csv(f"{games}/{games.split('/')[-1]}.csv", index=False)    
    # break

100%|██████████| 100/100 [01:01<00:00,  1.62it/s]


In [56]:
import pandas

dataa = pandas.read_csv("F:/video_classification/2017-05-20 - 21-45 Napoli 4 - 1 Fiorentina/2017-05-20 - 21-45 Napoli 4 - 1 Fiorentina.csv")
dataa[dataa['video_path'] == "[]"]

,chunk,video_base_dir,video_path,timestamp,labels,labels_encoder
45,45,F:/video_classification/2017-05-20 - 21-45 Nap...,[],45:00,"['Throw-in', 'Kick-off', 'Ball out of play']","1, 8, 9"


In [25]:

basedir = "F:/video_classification"
csv_files = [f"{basedir}/{i}/{i}.csv" for i in os.listdir(basedir)]

import pandas as pd
import glob

# Get all CSV files in the folder
# csv_files = glob.glob("path/to/folder/*.csv")  # Change to your folder path

# Read and concatenate all CSVs
df = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)
df = df[~df['video_path'].apply(lambda x: x == [])]

video_ids = []
for i in df['video_path']:
    ites = i.replace('[', '').replace(']', '')
    ites = ites.replace("'", '').replace("'", '')
    video_ids.append(normalize_path(ites))
df['video_id'] = video_ids


# Save the merged CSV
df.to_csv("./timesformer_dataset_v2.csv", index=False, sep = ";")

print("CSV files concatenated successfully!")


CSV files concatenated successfully!


In [28]:
data = {'video_id': df['video_id'], 'labels': df['labels_encoder']}

data = pd.DataFrame(data)
data.to_csv('timesformer_v2.csv',sep = ";", index=False)

In [51]:
import shutil
not_found_list = []
for i in tqdm(df['video_path']):
    try:
        checker = os.listdir(i)
    except FileNotFoundError:
        not_found_list.append(i)
        
    for file in checker:
        source_path = f"{i}/{file}"
        if os.path.isfile(f"{i}/{file}") and file.endswith(".mp4"):
            destination_visual_path = f"{i}/visual"
            os.makedirs(destination_visual_path, exist_ok = True)
            shutil.copy(source_path, f"{destination_visual_path}/{file}")
            os.remove(source_path)
            
        elif os.path.isfile(f"{i}/{file}") and (file.endswith(".wav") or file.endswith(".mp3")):
            destination_audio_path = f"{i}/audio"
            os.makedirs(destination_audio_path, exist_ok = True)
            shutil.copy(source_path, f"{destination_audio_path}/{file}")
            os.remove(source_path)
        else:
            print(f"File path {file} không nằm trong danh mục hỗ trợ. Thoát pressing ngay và luôn")
            # continue

  0%|          | 32/8034 [00:00<00:26, 307.53it/s]

File path frames không nằm trong danh mục hỗ trợ. Thoát pressing ngay và luôn
File path visual không nằm trong danh mục hỗ trợ. Thoát pressing ngay và luôn
File path frames không nằm trong danh mục hỗ trợ. Thoát pressing ngay và luôn
File path visual không nằm trong danh mục hỗ trợ. Thoát pressing ngay và luôn
File path frames không nằm trong danh mục hỗ trợ. Thoát pressing ngay và luôn
File path visual không nằm trong danh mục hỗ trợ. Thoát pressing ngay và luôn
File path frames không nằm trong danh mục hỗ trợ. Thoát pressing ngay và luôn
File path visual không nằm trong danh mục hỗ trợ. Thoát pressing ngay và luôn
File path frames không nằm trong danh mục hỗ trợ. Thoát pressing ngay và luôn
File path visual không nằm trong danh mục hỗ trợ. Thoát pressing ngay và luôn
File path frames không nằm trong danh mục hỗ trợ. Thoát pressing ngay và luôn
File path visual không nằm trong danh mục hỗ trợ. Thoát pressing ngay và luôn
File path frames không nằm trong danh mục hỗ trợ. Thoát pressing

  1%|▏         | 102/8034 [00:00<00:27, 285.88it/s]

File path frames không nằm trong danh mục hỗ trợ. Thoát pressing ngay và luôn
File path visual không nằm trong danh mục hỗ trợ. Thoát pressing ngay và luôn
File path frames không nằm trong danh mục hỗ trợ. Thoát pressing ngay và luôn
File path visual không nằm trong danh mục hỗ trợ. Thoát pressing ngay và luôn
File path frames không nằm trong danh mục hỗ trợ. Thoát pressing ngay và luôn
File path visual không nằm trong danh mục hỗ trợ. Thoát pressing ngay và luôn
File path frames không nằm trong danh mục hỗ trợ. Thoát pressing ngay và luôn
File path visual không nằm trong danh mục hỗ trợ. Thoát pressing ngay và luôn
File path frames không nằm trong danh mục hỗ trợ. Thoát pressing ngay và luôn
File path visual không nằm trong danh mục hỗ trợ. Thoát pressing ngay và luôn
File path frames không nằm trong danh mục hỗ trợ. Thoát pressing ngay và luôn
File path visual không nằm trong danh mục hỗ trợ. Thoát pressing ngay và luôn
File path frames không nằm trong danh mục hỗ trợ. Thoát pressing

  2%|▏         | 133/8034 [00:00<00:29, 270.95it/s]

...

File path frames không nằm trong danh mục hỗ trợ. Thoát pressing ngay và luôn
File path frames không nằm trong danh mục hỗ trợ. Thoát pressing ngay và luôn


100%|█████████▉| 8032/8034 [24:48<00:00,  6.34it/s]

File path frames không nằm trong danh mục hỗ trợ. Thoát pressing ngay và luôn
File path frames không nằm trong danh mục hỗ trợ. Thoát pressing ngay và luôn


100%|██████████| 8034/8034 [24:49<00:00,  5.40it/s]

File path frames không nằm trong danh mục hỗ trợ. Thoát pressing ngay và luôn


In [47]:
df = df[~df['video_path'].isin(not_found_list)]
df.to_csv("./timesformer_dataset_v2.csv", index=False, sep = ";")

In [46]:
len(df)

8034

In [53]:
import os
import pandas
import csv

basedir = "F:/video_classification"
for vdo in tqdm(os.listdir(basedir)):
    vdo_path = os.path.join(basedir, vdo)

    count_chunk = len([i for i in os.listdir(vdo_path) if os.path.isdir(f"{vdo_path}/{i}")])
    count_csv = len(pandas.read_csv(f"{vdo_path}/{vdo}.csv"))
    
    print([i for i in os.listdir(vdo_path) if os.path.isdir(f"{vdo_path}/{i}")])
    # if count_chunk > count_csv:
    #     count_remover = (count_chunk -1) - (count_csv -1)
    #     print("nooo")
    #     # for i in range(count_remover+1):
    #     #     if os.path.exists(f"{vdo_path}/chunk_{count_csv + i}"):
    #     #         shutil.rmtree(f"{vdo_path}/chunk_{count_csv + i}")
    # elif count_chunk == count_csv:
    #     print(f"Equally pass")
    #     continue
    # else:
    #     print(f"Mismatch size at {vdo} between chunker {count_chunk} and csv dataloader {count_csv}")
    #     continue

100%|██████████| 100/100 [00:00<00:00, 529.19it/s]

['chunk_0', 'chunk_1', 'chunk_10', 'chunk_11', 'chunk_12', 'chunk_13', 'chunk_14', 'chunk_15', 'chunk_16', 'chunk_17', 'chunk_18', 'chunk_19', 'chunk_2', 'chunk_20', 'chunk_21', 'chunk_22', 'chunk_23', 'chunk_24', 'chunk_25', 'chunk_26', 'chunk_27', 'chunk_28', 'chunk_29', 'chunk_3', 'chunk_30', 'chunk_31', 'chunk_32', 'chunk_33', 'chunk_34', 'chunk_35', 'chunk_36', 'chunk_37', 'chunk_38', 'chunk_39', 'chunk_4', 'chunk_40', 'chunk_41', 'chunk_42', 'chunk_43', 'chunk_44', 'chunk_45', 'chunk_46', 'chunk_47', 'chunk_48', 'chunk_49', 'chunk_5', 'chunk_50', 'chunk_51', 'chunk_52', 'chunk_53', 'chunk_54', 'chunk_55', 'chunk_56', 'chunk_57', 'chunk_58', 'chunk_59', 'chunk_6', 'chunk_60', 'chunk_61', 'chunk_62', 'chunk_63', 'chunk_64', 'chunk_65', 'chunk_66', 'chunk_67', 'chunk_68', 'chunk_69', 'chunk_7', 'chunk_70', 'chunk_71', 'chunk_72', 'chunk_73', 'chunk_74', 'chunk_75', 'chunk_76', 'chunk_77', 'chunk_78', 'chunk_79', 'chunk_8', 'chunk_80', 'chunk_81', 'chunk_82', 'chunk_83', 'chunk_84', 

In [11]:
basedir

'F:/video_classification'

In [51]:
os.path.exists("F:/video_classification/2016-02-03 - 22-45 Watford 0 - 0 Chelsea/chunk_45\frames\frame_0024.png")

False

In [3]:
file_path = "D:/projects/v2v/v5/data/timesformer_dataset_v2.csv"

# Đọc CSV (chỉnh delimiter nếu cần)
import pandas as pd
df = pd.read_csv(file_path, delimiter=";")  # Thay đổi delimiter nếu file dùng dấu khác

# Hiển thị vài dòng đầu tiên
df['new_video_path'][0]


'F:/video_classification/2014-11-04 - 20-00 Zenit Petersburg 1 - 2 Bayer Leverkusen/chunk_0/visual'

00:30 sẽ là 00:30  ----> 30 - 90
0031 sẽ là 00:30   ----> 31 - 90
01:29 sẽ là 00:30  ----> 89 - 90 1
01:35 sẽ là 01:30  ----> 95 - 90 1
01:42 sẽ là 01:30  ----> 102 - 90

02:31 sẽ là 01:30 --> 151 - 90 = 61
03:31 sẽ là 211 -90
05:55 sẽ là 04:30 --> 331 -

6:15